# Setup

In [1]:
# import libraries
import os
import sys
import time
import pandas as pd
import numpy as np
from scipy import stats
from scipy.interpolate import CubicSpline
import torch.optim as optim
import torch.nn as nn
import torch
from torch.optim import Adam

## Hyper-parameters

In [2]:
num_epochs = 30
batch_size = 64  # Set your batch size
learning_rate = 0.001

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# set the seed
np.random.seed(420)
torch.manual_seed(420)
torch.cuda.manual_seed(420)

# Load Data

In [4]:
# load data without header
data1 = pd.read_csv('./ISWC21_data_plus_raw/wetlab_data.csv')
# add header
data1.columns = ['subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity', 'activity_label_2']
data1.head()

,subject_id,acc_x,acc_y,acc_z,activity,activity_label_2
0,0,0.306563,9.196875,-1.22625,null_class,null_class
1,0,0.306563,9.196875,-1.22625,null_class,null_class
2,0,0.306563,9.196875,-1.22625,null_class,null_class
3,0,0.306563,9.196875,-1.22625,null_class,null_class
4,0,0.306563,9.196875,-1.22625,null_class,null_class


In [5]:
#remove activity label 2 column
data1 = data1.drop(['activity_label_2'], axis=1)
data1.shape

(3163679, 5)

In [6]:
#count number of unique subjects
print("Number of unique subjects: ", data1['subject_id'].nunique())

Number of unique subjects:  22


In [7]:
# load data without header
data2 = pd.read_csv('./ISWC21_data_plus_raw/rwhar_data.csv', header=None)
# add header
data2.columns = ['subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity']
data2.head()

,subject_id,acc_x,acc_y,acc_z,activity
0,0,-9.57434,-2.02733,1.34506,climbing_up
1,0,-9.56479,-1.99597,1.39345,climbing_up
2,0,-9.55122,-1.98445,1.41139,climbing_up
3,0,-9.51335,-1.97557,1.42615,climbing_up
4,0,-9.52959,-1.98187,1.45395,climbing_up


In [8]:
data2.shape

(3200803, 5)

In [9]:
#count number of unique subjects
print("Number of unique subjects: ", data2['subject_id'].nunique())

Number of unique subjects:  15


In [10]:
# load data without header
data3 = pd.read_csv('./ISWC21_data_plus_raw/sbhar_data.csv', header=None)
# add header
data3.columns = ['subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity']
data3.head()

,subject_id,acc_x,acc_y,acc_z,activity
0,0,0.443056,0.037500,0.888889,null_class
1,0,0.440278,0.041667,0.880556,null_class
2,0,0.451389,0.043056,0.876389,null_class
3,0,0.456944,0.034722,0.888889,null_class
4,0,0.447222,0.036111,0.888889,null_class


In [11]:
data3.shape

(1122772, 5)

In [12]:
#count number of unique subjects
print("Number of unique subjects: ", data3['subject_id'].nunique())

Number of unique subjects:  30


In [13]:
#print all of the unique subjects
print("Unique subjects: ", data3['subject_id'].unique())

Unique subjects:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]


In [14]:
# convert subject_id to int
data3['subject_id'] = data3['subject_id'].astype(int)

In [15]:
# # join all data in one dataframe row-wise
# data = pd.concat([data1, data2, data3], ignore_index=True, axis=0)
data = data3

In [16]:
data.shape

(1122772, 5)

In [17]:
#check null values in subject_id column
data['subject_id'].isnull().values.any()

False

# Data Preprocessing

## Split Train and Test Users

In [18]:
#split train and test data
#randomly select 20% of subjects for test data
test_subjects = data['subject_id'].unique()
test_subjects = np.random.choice(test_subjects, size=int(0.2*len(test_subjects)), replace=False)
# test_subjects = [ 9  7 26 29  1 24]
print("Test subjects: ", test_subjects)

#split data into train and test
train_data = data[~data['subject_id'].isin(test_subjects)]
test_data = data[data['subject_id'].isin(test_subjects)]
print("Train data shape: ", train_data.shape)
print("Test data shape: ", test_data.shape)

Test subjects:  [ 9  7 26 29  1 24]
Train data shape:  (897667, 5)
Test data shape:  (225105, 5)


## Data Normalization

In [19]:
# z normalization with respect to train data
train_data_mean = train_data[['acc_x', 'acc_y', 'acc_z']].mean()
train_data_std = train_data[['acc_x', 'acc_y', 'acc_z']].std()
# Normalize Training Data
train_data.loc[:, ['acc_x', 'acc_y', 'acc_z']] = (train_data[['acc_x', 'acc_y', 'acc_z']] - train_data_mean) / train_data_std

# Normalize Test Data with Training Statistics
test_data.loc[:, ['acc_x', 'acc_y', 'acc_z']] = (test_data[['acc_x', 'acc_y', 'acc_z']] - train_data_mean) / train_data_std

In [20]:
train_data_mean

acc_x    0.816012
acc_y   -0.007595
acc_z    0.074082
dtype: float64

In [21]:
train_data_std

acc_x    0.398664
acc_y    0.375481
acc_z    0.366527
dtype: float64

In [22]:
type(train_data_std)

pandas.core.series.Series

## Windowing

In [23]:
# function for sliding window

def sliding_window_samples(data, samples_per_window, overlap_ratio):
    """
    Return a sliding window measured in number of samples over a data array.

    :param data: input array, can be numpy or pandas dataframe
    :param samples_per_window: window length as number of samples
    :param overlap_ratio: overlap is meant as percentage and should be an integer value
    :return: tuple of windows and indices
    """
    windows = []
    indices = []
    curr = 0
    win_len = int(samples_per_window)
    if overlap_ratio is not None:
        overlapping_elements = int((overlap_ratio / 100) * (win_len))
        if overlapping_elements >= win_len:
            print('Number of overlapping elements exceeds window size.')
            return
    while curr < len(data) - win_len:
        windows.append(data[curr:curr + win_len])
        indices.append([curr, curr + win_len])
        curr = curr + win_len - overlapping_elements
    try:
        result_windows = np.array(windows)
        result_indices = np.array(indices)
    except:
        result_windows = np.empty(shape=(len(windows), win_len, data.shape[1]), dtype=object)
        result_indices = np.array(indices)
        for i in range(0, len(windows)):
            result_windows[i] = windows[i]
            result_indices[i] = indices[i]
    return result_windows, result_indices

In [24]:
# sampling_rate = 50
# time_window = 8
# window_size = sampling_rate * time_window
# overlap_ratio = 50

sampling_rate = 50
time_window = 2
window_size = sampling_rate * time_window
overlap_ratio = 0

train_window_data, _ = sliding_window_samples(train_data, window_size, overlap_ratio)
print(f"shape of train window dataset ({time_window} sec with {overlap_ratio}% overlap): {train_window_data.shape}")

test_window_data, _ = sliding_window_samples(test_data, window_size, overlap_ratio)
print(f"shape of test window dataset ({time_window} sec with {overlap_ratio}% overlap): {test_window_data.shape}")

shape of train window dataset (2 sec with 0% overlap): (8976, 100, 5)
shape of test window dataset (2 sec with 0% overlap): (2251, 100, 5)


In [25]:
train_window_data[0]

array([[0, -0.9355155521152527, 0.12009837894797139, 2.223044603166194,
        'null_class'],
       [0, -0.9424832899580327, 0.13119524723936618, 2.200308651740837,
        'null_class'],
       [0, -0.9146124605540366, 0.1348942033364978, 2.188940676028158,
        'null_class'],
       [0, -0.9006769848684767, 0.11270046675370819, 2.223044603166194,
        'null_class'],
       [0, -0.9250639453510827, 0.11639942285083979, 2.223044603166194,
        'null_class'],
       [0, -0.9285478142724728, 0.14229211553076074, 2.188940676028158,
        'null_class'],
       [0, -0.9041608537898667, 0.1348942033364978, 2.2268339284037535,
        'null_class'],
       [0, -0.9041608537898667, 0.1348942033364978, 2.2268339284037535,
        'null_class'],
       [0, -0.9111285916326465, 0.14229211553076074, 2.2457802892698013,
        'null_class'],
       [0, -0.9111285916326465, 0.14229211553076074, 2.2457802892698013,
        'null_class'],
       [0, -0.9111285916326465, 0.123797335045103

## Get Only the Accelerometer

In [26]:
# remove the label column
train_window_data = train_window_data[:, :, :-1]
# train_window_data = train_window_data[:, :, :-1]
#remove the subject column
train_window_data = train_window_data[:, :, 1:]

test_window_data = test_window_data[:, :, :-1]
test_window_data = test_window_data[:, :, 1:]


In [27]:
train_window_data[0].shape

(100, 3)

In [28]:
test_window_data[0].shape

(100, 3)

# Data Transformation

In [29]:
def add_jitter(data, noise_factor=0.05):
    jitter = noise_factor * np.random.randn(*data.shape)
    return data + jitter

In [30]:
def scale_data(data, min_scale=0.5, max_scale=1.5):
    scaling_factor = np.random.uniform(min_scale, max_scale)
    return data * scaling_factor


In [31]:
def rotate_data(data):
    # Invert the sign of the data to simulate sensor rotation
    return -data

In [32]:
def negate_data(data):
    return -data

In [33]:
def horizontal_flip(data):
    # This function now correctly handles 2D data arrays
    return data[::-1, :]

In [34]:
def permute_data(data, num_segments=4):
    segment_length = data.shape[0] // num_segments  # Adjusted to the first dimension for 2D data
    permuted_indices = np.random.permutation(num_segments)
    return np.concatenate(
        [data[segment_length * idx:segment_length * (idx + 1), :] for idx in permuted_indices], axis=0)  # Concatenating along the first axis

In [35]:
from scipy.interpolate import interp1d
import numpy as np

def time_warp(data, warp_factor_range=(0.8, 1.2)):
    sequence_length, num_channels = data.shape
    original_time_points = np.linspace(0, 1, sequence_length)
    warp_factor = np.random.uniform(*warp_factor_range)
    
    # Generate new time points based on the warp factor
    warped_time_points = np.linspace(0, warp_factor, sequence_length)

    warped_data = np.zeros_like(data)
    for j in range(num_channels):
        # Interpolate each channel
        interpolation = interp1d(original_time_points, data[:, j], 
                                 kind='linear', fill_value="extrapolate")
        warped_data[:, j] = interpolation(warped_time_points)

    return warped_data


In [36]:
def shuffle_channels(data):
    # Assuming data is 2D with shape (sequence_length, num_channels)
    shuffled_indices = np.random.permutation(data.shape[1])  # Shuffle along the second dimension
    return data[:, shuffled_indices]

## Transform Training Data

In [37]:
import numpy as np

# Initialize lists to store datasets
train_dataset = [[] for _ in range(8)]

# Loop over all training data
for data in train_window_data:
    # loop over all transformations
    # print(f"shape of data: {data.shape}")
    data_array = np.array(data, dtype=np.float32)
    for j in range(8):
        # Original data with label 0
        train_dataset[j].append((data_array, 0))
        # Apply transformation based on j and save it in the transformed_data variable
        if j == 0:
            transformed_data = add_jitter(data_array)
        elif j == 1:
            transformed_data = scale_data(data_array)
        elif j == 2:
            transformed_data = rotate_data(data_array)
        elif j == 3:
            transformed_data = negate_data(data_array)
        elif j == 4:
            transformed_data = horizontal_flip(data_array)
        elif j == 5:
            transformed_data = permute_data(data_array)
        elif j == 6:
            transformed_data = time_warp(data_array)
        elif j == 7:
            transformed_data = shuffle_channels(data_array)
        # Append the transformed data with label 1
        transformed_data_array = np.array(transformed_data, dtype=np.float32)
        train_dataset[j].append((transformed_data_array, 1))

for dataset in train_dataset:
    shapes = set(tuple(d.shape) for d, _ in dataset)
    if len(shapes) > 1:
        print("Inconsistent shapes found in dataset:", shapes)

# Convert lists to numpy arrays
for j in range(8):
    data, labels = zip(*train_dataset[j])
    data = np.array(data)
    labels = np.array(labels)
    train_dataset[j] = (data, labels)


In [38]:
train_dataset[1][0]

array([[[-0.9355155 ,  0.12009838,  2.2230446 ],
        [-0.9424833 ,  0.13119525,  2.2003086 ],
        [-0.9146125 ,  0.1348942 ,  2.1889408 ],
        ...,
        [-0.58016163, -2.3360088 ,  0.10102688],
        [-0.49654904, -2.395192  ,  0.11997344],
        [-0.4233879 , -2.3692992 , -0.40674165]],

       [[-1.0421286 ,  0.13378501,  2.4763868 ],
        [-1.0498904 ,  0.14614649,  2.4510598 ],
        [-1.0188434 ,  0.15026699,  2.4383965 ],
        ...,
        [-0.6462779 , -2.6022246 ,  0.11254008],
        [-0.55313665, -2.6681523 ,  0.13364583],
        [-0.47163796, -2.6393087 , -0.45309466]],

       [[-0.13074353, -2.228739  , -0.52421045],
        [ 0.14796537, -2.6060326 , -0.7288336 ],
        [-0.23177561, -3.0979936 , -1.3275458 ],
        ...,
        [ 0.31519058, -0.80094177, -0.16422534],
        [ 0.36744863, -0.9045125 , -0.16422534],
        [ 0.510287  , -0.89711463, -0.27411553]],

       ...,

       [[ 0.29202315, -0.3310534 , -0.38105655],
        [ 0

In [39]:
# print the shape of all training datasets
for j in range(8):
    print(f"shape of training dataset {j}: {train_dataset[j][0].shape}")

shape of training dataset 0: (17952, 100, 3)
shape of training dataset 1: (17952, 100, 3)
shape of training dataset 2: (17952, 100, 3)
shape of training dataset 3: (17952, 100, 3)
shape of training dataset 4: (17952, 100, 3)
shape of training dataset 5: (17952, 100, 3)
shape of training dataset 6: (17952, 100, 3)
shape of training dataset 7: (17952, 100, 3)


In [40]:
# print the class distribution of all training datasets
for j in range(8):
    print(f"Class distribution of training dataset {j}: {np.unique(train_dataset[j][1], return_counts=True)}")

Class distribution of training dataset 0: (array([0, 1]), array([8976, 8976], dtype=int64))
Class distribution of training dataset 1: (array([0, 1]), array([8976, 8976], dtype=int64))
Class distribution of training dataset 2: (array([0, 1]), array([8976, 8976], dtype=int64))
Class distribution of training dataset 3: (array([0, 1]), array([8976, 8976], dtype=int64))
Class distribution of training dataset 4: (array([0, 1]), array([8976, 8976], dtype=int64))
Class distribution of training dataset 5: (array([0, 1]), array([8976, 8976], dtype=int64))
Class distribution of training dataset 6: (array([0, 1]), array([8976, 8976], dtype=int64))
Class distribution of training dataset 7: (array([0, 1]), array([8976, 8976], dtype=int64))


## Transform Testing Data

In [41]:
# initialize lists to store datasets for test data
test_dataset = [[] for _ in range(8)]

# loop over all test data
for data in test_window_data:
    data_array = np.array(data, dtype=np.float32)
    # loop over all transformations
    for j in range(8):
        # Original data with label 0
        test_dataset[j].append((data_array, 0))
        # Apply transformation based on j and save it in the transformed_data variable
        if j == 0:
            transformed_data = add_jitter(data_array)
        elif j == 1:
            transformed_data = scale_data(data_array)
        elif j == 2:
            transformed_data = rotate_data(data_array)
        elif j == 3:
            transformed_data = negate_data(data_array)
        elif j == 4:
            transformed_data = horizontal_flip(data_array)
        elif j == 5:
            transformed_data = permute_data(data_array)
        elif j == 6:
            transformed_data = time_warp(data_array)
        elif j == 7:
            transformed_data = shuffle_channels(data_array)
        # Append the transformed data with label 1
        transformed_data_array = np.array(transformed_data, dtype=np.float32)
        test_dataset[j].append((transformed_data_array, 1))

# check for inconsistent shapes
for dataset in test_dataset:
    shapes = set(tuple(d.shape) for d, _ in dataset)
    if len(shapes) > 1:
        print("Inconsistent shapes found in dataset:", shapes)

# Convert lists to numpy arrays
for j in range(8):
    data, labels = zip(*test_dataset[j])
    data = np.array(data)
    labels = np.array(labels)
    test_dataset[j] = (data, labels)

In [42]:
# print the shape of all test datasets
for j in range(8):
    print(f"shape of test dataset {j}: {test_dataset[j][0].shape}")

shape of test dataset 0: (4502, 100, 3)
shape of test dataset 1: (4502, 100, 3)
shape of test dataset 2: (4502, 100, 3)
shape of test dataset 3: (4502, 100, 3)
shape of test dataset 4: (4502, 100, 3)
shape of test dataset 5: (4502, 100, 3)
shape of test dataset 6: (4502, 100, 3)
shape of test dataset 7: (4502, 100, 3)


In [43]:
# print the class distribution of all test datasets
for j in range(8):
    print(f"Class distribution of test dataset {j}: {np.unique(test_dataset[j][1], return_counts=True)}")

Class distribution of test dataset 0: (array([0, 1]), array([2251, 2251], dtype=int64))
Class distribution of test dataset 1: (array([0, 1]), array([2251, 2251], dtype=int64))
Class distribution of test dataset 2: (array([0, 1]), array([2251, 2251], dtype=int64))
Class distribution of test dataset 3: (array([0, 1]), array([2251, 2251], dtype=int64))
Class distribution of test dataset 4: (array([0, 1]), array([2251, 2251], dtype=int64))
Class distribution of test dataset 5: (array([0, 1]), array([2251, 2251], dtype=int64))
Class distribution of test dataset 6: (array([0, 1]), array([2251, 2251], dtype=int64))
Class distribution of test dataset 7: (array([0, 1]), array([2251, 2251], dtype=int64))


# Create the Dataloader

In [44]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        # Convert data and labels to PyTorch tensors
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)  # Assuming labels are integers

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Initialize DataLoader for each dataset
train_loaders = []

for j in range(8):
    # Assuming train_dataset[j][0] is the data and train_dataset[j][1] are the labels
    data, labels = train_dataset[j]
    transformed_dataset = CustomDataset(data, labels)
    train_loader = DataLoader(transformed_dataset, batch_size=batch_size, shuffle=True)
    train_loaders.append(train_loader)

In [45]:
# creating test loaders

test_loaders = []
for j in range(8):
    test_data, test_labels = test_dataset[j]
    test_transformed_dataset = CustomDataset(test_data, test_labels)
    test_loader = DataLoader(test_transformed_dataset, batch_size=batch_size, shuffle=False)  # Usually, we don't shuffle test data
    test_loaders.append(test_loader)

# Model Architecture

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# need to reshape and transpose the data to fit the input shape of the model

class TPN(nn.Module):
    def __init__(self):
        super(TPN, self).__init__()
        self.trunk = nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=32, kernel_size=24, stride=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=16, stride=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Conv1d(in_channels=64, out_channels=96, kernel_size=8, stride=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.AdaptiveMaxPool1d(output_size=1)
        )

        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(96, 256),
                nn.ReLU(),
                nn.Linear(256, 1),
                nn.Sigmoid()
            ) for _ in range(8)  # 8 heads for 8 different transformations
        ])

    def forward(self, x):
        x = self.trunk(x)
        x = x.view(x.size(0), -1)  # Flatten the output for the fully-connected layer
        outputs = [head(x) for head in self.heads]
        return outputs


# Training

In [47]:
# model = TPN()
# optimizer = optim.Adam(model.parameters(), lr=0.0003)
# criterion = nn.BCELoss()

# # training loop
# for epoch in range(30):
#     print(f"Epoch {epoch + 1}")
#     for j in range(8):
#         train_loss = 0
#         model.train()
#         for i, (data, labels) in enumerate(train_loaders[j]):
#             optimizer.zero_grad()
#             outputs = model(data.transpose(1, 2))  # Transpose the data to fit the input shape of the model
#             loss = criterion(outputs[j].squeeze(), labels.float())  # Squeeze the output of the model to fit the loss function
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()
#         print(f"Training loss of dataset {j}: {train_loss / len(train_loaders[j])}")

In [48]:
# model = TPN()
# optimizer = Adam(model.parameters(), lr=0.0003)
# criterion = nn.BCELoss()

# # training loop
# for epoch in range(num_epochs):
#     print(f"Epoch {epoch + 1}")
#     model.train()
#     total_loss = 0

#     for data, labels in train_loader:  # Assuming train_loader is a combined DataLoader for all tasks
#         optimizer.zero_grad()
#         outputs = model(data.transpose(1, 2))  # Transpose data to match the model's expected input shape
#         print(f"input shape: {data.transpose(1, 2).shape}")
#         print(f"output length: {len(outputs)}")
#         print(f"output shape: {outputs[0].shape}")
#         print(f"label shape: {labels[j].shape}")
#         print(f"labels: {labels[j]}")
#         loss = 0
#         for j in range(8):  # Assuming 8 tasks
#             task_loss = criterion(outputs[j].squeeze(), labels[j].float())
#             loss += task_loss

#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()

#     avg_loss = total_loss / len(train_loader)
#     print(f"Average Training Loss: {avg_loss}")


In [49]:
# model = TPN()
# optimizer = Adam(model.parameters(), lr=0.0003)
# criterion = nn.BCELoss()

# for epoch in range(num_epochs):
#     print(f"Epoch {epoch + 1}")
#     model.train()
#     total_loss = 0

#     # Assuming all DataLoader have the same length
#     for batch in zip(*train_loaders):
#         optimizer.zero_grad()
#         loss = 0

#         for j, (data, labels) in enumerate(batch):
#             data = data.transpose(1, 2)  # Transpose to match input shape
#             outputs = model(data)
#             task_loss = criterion(outputs[j].squeeze(), labels.float())
#             loss += task_loss

#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()

#     avg_loss = total_loss / len(train_loaders[0])  # Average over batches
#     print(f"Average Training Loss: {avg_loss}")


In [50]:
import torch.optim as optim
import numpy as np

model = TPN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0003, weight_decay=0.0001)
criterion = nn.BCELoss()

# Early stopping parameters
patience = 5  # Number of epochs to wait for improvement before stopping
min_delta = 0.001  # Minimum change to qualify as an improvement
best_loss = np.inf  # Initialize best loss to infinity
counter = 0  # Initialize counter for early stopping

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}")
    model.train()
    total_loss = 0

    for batch in zip(*train_loaders):
        optimizer.zero_grad()
        loss = 0

        for j, (data, labels) in enumerate(batch):
            data = data.transpose(1, 2)  # Transpose to match input shape
            data = data.to(device)
            labels = labels.to(device)
            outputs = model(data)
            task_loss = criterion(outputs[j].squeeze(), labels.float())
            loss += task_loss

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loaders[0])
    print(f"Average Training Loss: {avg_loss}")

    # Validation phase
    model.eval()
    validation_loss = 0
    with torch.no_grad():
        for data, labels in test_loader:
            data = data.transpose(1, 2)
            data = data.to(device)
            labels = labels.to(device)
            outputs = model(data)
            val_loss = criterion(outputs[0].squeeze(), labels.float())  # Assuming single task validation
            validation_loss += val_loss.item()

    avg_val_loss = validation_loss / len(test_loader)
    print(f"Average Validation Loss: {avg_val_loss}")

    # Early stopping check
    if best_loss - avg_val_loss > min_delta:
        best_loss = avg_val_loss
        counter = 0  # Reset counter if validation loss improved
    else:
        counter += 1  # Increment counter if no improvement
        if counter >= patience:
            print(f"Early stopping triggered after {epoch + 1} epochs.")
            break  # Stop training if no improvement for 'patience' consecutive epochs

Epoch 1
Average Training Loss: 4.231698088798659
Average Validation Loss: 0.7221342602246245
Epoch 2
Average Training Loss: 3.3938948980854073
Average Validation Loss: 0.6741924504159202
Epoch 3
Average Training Loss: 3.222850152181985
Average Validation Loss: 0.6776066926163686
Epoch 4
Average Training Loss: 3.1367480152442364
Average Validation Loss: 0.6851188792309291
Epoch 5
Average Training Loss: 3.0761950525100543
Average Validation Loss: 0.6709481620452773
Epoch 6
Average Training Loss: 3.0403219652345594
Average Validation Loss: 0.6652392921313434
Epoch 7
Average Training Loss: 3.003853150534036
Average Validation Loss: 0.6602110081995037
Epoch 8
Average Training Loss: 2.9832617082629764
Average Validation Loss: 0.6531092474158381
Epoch 9
Average Training Loss: 2.952975748272553
Average Validation Loss: 0.654988662457802
Epoch 10
Average Training Loss: 2.9339883853532243
Average Validation Loss: 0.6538829241000431
Epoch 11
Average Training Loss: 2.914205585086049
Average Valida

# Testing

In [51]:
import torch
from sklearn.metrics import accuracy_score, f1_score

model.eval()  # Set the model to evaluation mode

# Initialize lists to store metrics for each task
accuracies = []
f1_scores = []

with torch.no_grad():  # No need to track gradients during evaluation
    for j, test_loader in enumerate(test_loaders):
        all_labels = []
        all_predictions = []

        for data, labels in test_loader:
            data = data.transpose(1, 2)  # Transpose data if necessary
            data = data.to(device)
            labels = labels.to(device)
            outputs = model(data)
            predictions = torch.round(outputs[j].squeeze())  # Convert to binary predictions

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

        # Calculate metrics for the current task
        acc = accuracy_score(all_labels, all_predictions)
        f1 = f1_score(all_labels, all_predictions, average='binary')

        accuracies.append(acc)
        f1_scores.append(f1)

        print(f"Task {j} - Accuracy: {acc}, F1-Score: {f1}")

# You can also calculate the average metrics across tasks, if needed
avg_accuracy = sum(accuracies) / len(accuracies)
avg_f1_score = sum(f1_scores) / len(f1_scores)
print(f"Average Accuracy: {avg_accuracy}, Average F1-Score: {avg_f1_score}")

Task 0 - Accuracy: 0.5370946246112839, F1-Score: 0.5370946246112839
Task 1 - Accuracy: 0.9153709462461128, F1-Score: 0.9091775923718712
Task 2 - Accuracy: 0.9946690359840071, F1-Score: 0.9946690359840071
Task 3 - Accuracy: 0.9946690359840071, F1-Score: 0.9946690359840071
Task 4 - Accuracy: 0.7072412261217237, F1-Score: 0.727798430400661
Task 5 - Accuracy: 0.7159040426477121, F1-Score: 0.6882768705825006
Task 6 - Accuracy: 0.5944024877832075, F1-Score: 0.5379554655870445
Task 7 - Accuracy: 0.8636161705908485, F1-Score: 0.85068093385214
Average Accuracy: 0.7903709462461128, Average F1-Score: 0.7800402486716894


In [52]:
# 20 epoch; Average Accuracy: 0.8046596975088968, Average F1-Score: 0.7754674519258395
# 30 epoch; Average Accuracy: 0.8065502669039145, Average F1-Score: 0.7555106030018032
# 30 epoch, 2 sec window; Average Accuracy: 0.7903709462461128, Average F1-Score: 0.7800402486716894

In [54]:
# save the model
torch.save(model.state_dict(), './multitask/tpn_30_epoch_regularized_2.pt')